In [13]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from sklearn.preprocessing import StandardScaler

#Lettura dei file csv e selezione delle colonne di interesse
df1 = pd.read_csv('C:\\Users\\Utente\\Desktop\\cakio\\dcereijo-player-scores\\data\\uefarankteam3.csv')
df2 = pd.read_csv('C:\\Users\\Utente\\Desktop\\cakio\\dcereijo-player-scores\\data\\spi_global_rankings.csv')

#Esecuzione del matching tramite la funzione extractOne di fuzzywuzzy
df_merged = pd.merge(df2, df1, how='cross')
df_merged['correct_ratio'] = df_merged.apply(lambda x: fuzz.token_set_ratio(x['name22'], x['nome squadra']) if str(x['name22']) != 'nan' and str(x['nome squadra']) != 'nan' else 0, axis=1)
df_results = df_merged[df_merged['correct_ratio'] >= 95]

######Stampa dei risultati
print(df_results)

#######Seleziona solo le righe relative ai paesi desiderati
desired_countries = ['Ger', 'Eng', 'Ita', 'Esp', 'Fra', 'Ned', 'Por', 'Rus']
df3 = df_results[df_results['paese'].isin(desired_countries)]

#####Normalizza i punteggi di ogni variabile
scaler = StandardScaler()
df3[['off', 'def', 'spi', 'UEFAPoints']] = scaler.fit_transform(df3[['off', 'def', 'spi', 'UEFAPoints']])

####Assegna un peso diverso a ciascuna variabile
weights = {'off': 0.2, 'def': 0.2, 'spi': 0.2, 'UEFAPoints': 0.4}

###Calcola il punteggio overall per ogni squadra
df3['Overall'] = (df3['off']*weights['off'] + df3['def']*weights['def'] +
df3['spi']*weights['spi'] + df3['UEFAPoints']*weights['UEFAPoints'])

##Normalizza il punteggio overall in modo che vari da 0 a 1
df3['Overall'] = (df3['Overall'] - df3['Overall'].min()) / (df3['Overall'].max() - df3['Overall'].min())

#Stampa il dataframe risultante
print(df3)

##Raggruppa il dataframe per la colonna 'league' e calcola la somma degli overall
league_overall = df3.groupby('league')['Overall'].sum()
print(league_overall)





        rank  prev_rank            name22                         league  \
4          1          1   Manchester City        Barclays Premier League   
568        2          2     Bayern Munich              German Bundesliga   
1134       3          4       Real Madrid       Spanish Primera Division   
1703       4          3         Barcelona       Spanish Primera Division   
2275       5          6         Liverpool        Barclays Premier League   
...      ...        ...               ...                            ...   
297924   526        528  Asteras Tripolis             Greek Super League   
303513   536        526       KV Oostende         Belgian Jupiler League   
304730   538        538          Giannina             Greek Super League   
305881   540        540   Shamrock Rovers  UEFA Europa Conference League   
334847   591        584    Brommapojkarna            Swedish Allsvenskan   

         off   def    spi  Unnamed: 0  posizione  nn  ... 17/18 18/19  19/20  \
4      

C:\Users\Utente\anaconda4\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
<ipython-input-13-435360b2a039>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Overall'] = (df3['off']*weights['off'] + df3['def']*weights['def'] +
<ipython-input-13-435360b2a039>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [14]:
##Raggruppa il dataframe per la colonna 'league' e calcola la somma delle squadre
league_count = df3.groupby('league')['name22'].count()
print(league_count)

league
Argentina Primera Division      1
Barclays Premier League        12
Dutch Eredivisie                9
English League Championship     4
French Ligue 1                  9
French Ligue 2                  2
German Bundesliga              13
Italy Serie A                  12
Portuguese Liga                10
Russian Premier Liga            7
Spanish Primera Division       12
Spanish Segunda Division        2
Name: name22, dtype: int64


In [15]:
# Calcola la media tra il valore di Overall delle leghe e la quantità di squadre
league_average = league_overall / league_count

# Stampa il risultato
print(league_average)


league
Argentina Primera Division     0.349936
Barclays Premier League        0.484012
Dutch Eredivisie               0.354114
English League Championship    0.088946
French Ligue 1                 0.353717
French Ligue 2                 0.098041
German Bundesliga              0.434149
Italy Serie A                  0.397451
Portuguese Liga                0.240979
Russian Premier Liga           0.256131
Spanish Primera Division       0.437027
Spanish Segunda Division       0.195490
dtype: float64


In [20]:
#Lettura dei file csv e selezione delle colonne di interesse
df6 = pd.read_csv('C:\\Users\\Utente\\Desktop\\cakio\\dcereijo-player-scores\\data\\LEAGUEmean1.csv')

In [22]:
# Normalizza il punteggio overall in modo che vari da 0 a 1
df6['mean_overall'] = (df6['mean_overall'] - df6['mean_overall'].min()) / (df6['mean_overall'].max() - df6['mean_overall'].min())
# Stampa il nuovo dataframe
print(df6)



                     league  mean_overall competiton_id
0   Barclays Premier League      0.716867           GB1
1          Dutch Eredivisie      0.333710           NL1
2            French Ligue 1      0.332541           FR1
3         German Bundesliga      0.569787            L1
4             Italy Serie A      0.461541           IT1
5           Portuguese Liga      0.000000           PO1
6      Russian Premier Liga      0.044693           RU1
7  Spanish Primera Division      0.578278           SP1
8     UEFA Champions League      1.000000            CL
9        UEFA Europa League      0.439563            EL
